In [1]:
import os, sys, logging, importlib, math, time, subprocess
import rasterio, affine

import geopandas as gpd
import pandas as pd

from zipfile import ZipFile
from affine import Affine
from rasterio import features
from rasterio.mask import mask
from rasterio.features import rasterize

import GEP

'''prints the time along with the message'''
def tPrint(s):
    print("%s\t%s" % (time.strftime("%H:%M:%S"), s))

# Mapping electrification
Code to download results from S3

We need to convert certain fields in the results to a raster; these fields need to be calculated 

1. Population already electrified 2030  
    a. Rasterize field **ElecPop**
2. New population electrified by grid 2030  
    a. Filter data where **FinalElecCode2030** == 1.0  
    b. Set **NewConnections2025** = 0 where **FinalElecCode2025** == 99.0  
    c. Combine **NewConnections2030** with **NewConnections2025**  
3. New population electrified by off grid 2030  
    a. Filter data where **FinalElecCode2030** != 1.0  
    b. Set **NewConnections2025** = 0 where **FinalElecCode2025** != **FinalElecCode2030**  
    c. Combine **NewConnections2030** with **NewConnections2025**  

In [ ]:
# delete all existing rasterized results
'''all_rasters = []
for root, dirs, files in os.walk('/media/gost/DATA1/GEP/Clusters'):
    for f in files:
        if "rasterized_gep" in f:
            all_rasters.append(os.path.join(root, f))
for f in all_rasters:
    os.remove(f)'''

In [2]:
resolution = 0.01 #Resolution of output raster in degrees
elecRate = 0.90   #settlement electrification rate to be considered electrified
year = 2030       #year of analysis 
cluster_folder = '/media/gost/DATA1/GEP/Clusters'
scenario_base = '/media/gost/DATA1/GEP/Scenarios'
raster_folder = '/media/gost/DATA1/GEP/rasterized_countries'
vrt_folder = '/media/gost/DATA1/GEP/GEP_VRTs'


In [3]:
importlib.reload(GEP)
scenario = "0_0_0_0_1_0"
scenario_folder = os.path.join(scenario_base, f'{scenario}')
# Identify countries to process
zipFiles = []
processedCountries = []
for root, folders, files in os.walk(scenario_folder):    
    processedCountries.append([f for f in folders])
    for f in files:
        if f[-4:] == ".csv":
            zipFiles.append(f[:-4])
processedCountries = [item[:4] for item in zipFiles]
zipFiles #list of country/scenarios to process

['gh-2-0_0_0_0_1_0',
 'mw-2-0_0_0_0_1_0',
 'ao-2-0_0_0_0_1_0',
 'bd-2-0_0_0_0_1_0',
 'bf-2-0_0_0_0_1_0',
 'bi-2-0_0_0_0_1_0',
 'bj-2-0_0_0_0_1_0',
 'bw-2-0_0_0_0_1_0',
 'cd-2-0_0_0_0_1_0',
 'cf-2-0_0_0_0_1_0',
 'cg-2-0_0_0_0_1_0',
 'ci-2-0_0_0_0_1_0',
 'cm-2-0_0_0_0_1_0',
 'dj-2-0_0_0_0_1_0',
 'er-2-0_0_0_0_1_0',
 'et-2-0_0_0_0_1_0',
 'fm-2-0_0_0_0_1_0',
 'ga-2-0_0_0_0_1_0',
 'gm-2-0_0_0_0_1_0',
 'gn-2-0_0_0_0_1_0',
 'gq-2-0_0_0_0_1_0',
 'gw-2-0_0_0_0_1_0',
 'hn-2-0_0_0_0_1_0',
 'ht-2-0_0_0_0_1_0',
 'ke-2-0_0_0_0_1_0',
 'kh-2-0_0_0_0_1_0',
 'km-2-0_0_0_0_1_0',
 'lr-2-0_0_0_0_1_0',
 'ls-2-0_0_0_0_1_0',
 'mg-2-0_0_0_0_1_0',
 'ml-2-0_0_0_0_1_0',
 'mm-2-0_0_0_0_1_0',
 'mr-2-0_0_0_0_1_0',
 'mz-2-0_0_0_0_1_0',
 'na-2-0_0_0_0_1_0',
 'ne-2-0_0_0_0_1_0',
 'ng-2-0_0_0_0_1_0',
 'ni-2-0_0_0_0_1_0',
 'pg-2-0_0_0_0_1_0',
 'pk-2-0_0_0_0_1_0',
 'rw-2-0_0_0_0_1_0',
 'sb-2-0_0_0_0_1_0',
 'sd-2-0_0_0_0_1_0',
 'sl-2-0_0_0_0_1_0',
 'sn-2-0_0_0_0_1_0',
 'so-2-0_0_0_0_1_0',
 'ss-2-0_0_0_0_1_0',
 'st-2-0_0_0_

In [4]:
importlib.reload(GEP)
# Loop through all countries and rasterize
minPop = 100
for country in processedCountries:
    xx_so = GEP.gep_results(country[:4], 
                scenariosFolder=scenario_base,
                clustersFolder = cluster_folder,
                scenario=scenario)
    break
    outFile = os.path.join(raster_folder, f"{country[:2]}_rasterized_gep_{resolution}_{scenario}_{year}_gt{minPop}.tif")    
    if not os.path.exists(outFile):
        tPrint(f"Processing {country}")    
        res = xx_so.join_results()
        res = res.loc[res['Pop2030'] > minPop,]
        # Identify already electrified settlements and set value to 98
        res['already_electrified'] = (res['ElecPopCalib'] / res['Pop2020']) > elecRate
        res.loc[res['already_electrified'] == 1, 'FinalElecCode%s' % year] = 98
        xx_so.rasterize_results(res, outFile, res=resolution, field='FinalElecCode%s' % year)
    else:
        tPrint(f"{country} already rasterized")    

In [ ]:
### Run a series of GDAL commands to create No Data rasters and stack them
command_file = os.path.join(vrt_folder, f'create_vrt_{resolution}_{scenario}_{year}.sh')
with open(command_file, 'w') as vrt_file:
    vrt_file.write('#!/bin/sh \n')
    # Convert 0 in data to no data
    outFolder = raster_folder
    allImages = []
    for root, folders, files in os.walk(outFolder):
        for f in files:
            if "%s_%s_%s_gt%s.tif" % (resolution, scenario, year, minPop) in f and f.endswith(".tif"):
                allImages.append(os.path.join(root, f))

    for inFile in allImages:
        outFile = inFile.replace(".tif", ".tif")
        vrt_file.write(f'gdal_translate -of GTiff -a_nodata 0 {inFile} {outFile}')
        vrt_file.write("\n")
    vrt_file.write(f'find {raster_folder} -name "*{resolution}_{scenario}_{year}_gt{minPop}_noData.tif" > tifFiles.txt')
    vrt_file.write("\n")
    vrt_file.write(f"gdalbuildvrt -input_file_list tifFiles.txt {vrt_folder}/combined_gep_rasterized_{resolution}_{scenario}_{year}.vrt")
    vrt_file.write("\n")
subprocess.call(command_file)

# Mapping annual electrification breakdown
The GEP results contain information describing the electrification status in 2020, 2025, and 2030 (ElecStart,ElecStatusIn2025, ElecStatusIn2030). Based on these we can separate the dataset into three groups:

1. Electrified
2. Electrified first - electrified between 2020 and 2025
3. Electrified second - electrified between 2025 and 2030

For the second and third groups, we can approximate the year they were electrified in that range based on other variables. In the example below, we use the distance to MV lines, under the assumption that places closer to MV lines will be electrified first.

The code works by take the two groups separately, sorting them according the the predictive variable, and then assigning the year of electrification sequentially.

In [152]:
def get_years(shp, start_year, end_year):
    '''    
        Get a list of numbers between start_year and end_year of length shp    
    '''
    n_years = round(shp/(end_year - start_year + 1))
    yr = [x for x in range(start_year, end_year + 1) for _ in range(n_years)]
    if len(yr) > shp:
        return(yr[:shp])
    while len(yr) < shp:
        yr.append(end_year)        
    return(yr)

get_years(10, 2, 6)

[2, 2, 3, 3, 4, 4, 5, 5, 6, 6]

In [153]:
# Get a list of all countries for the selected scenario
importlib.reload(GEP)
scenario = "0_0_0_0_1_0"
scenario_folder = os.path.join(scenario_base, f'{scenario}')
# Identify countries to process
zipFiles = []
processedCountries = []
for root, folders, files in os.walk(scenario_folder):    
    processedCountries.append([f for f in folders])
    for f in files:
        if f[-4:] == ".csv":
            zipFiles.append(f[:-4])
processedCountries = [item[:4] for item in zipFiles]
zipFiles #list of country/scenarios to process

['gh-2-0_0_0_0_1_0',
 'mw-2-0_0_0_0_1_0',
 'ao-2-0_0_0_0_1_0',
 'bd-2-0_0_0_0_1_0',
 'bf-2-0_0_0_0_1_0',
 'bi-2-0_0_0_0_1_0',
 'bj-2-0_0_0_0_1_0',
 'bw-2-0_0_0_0_1_0',
 'cd-2-0_0_0_0_1_0',
 'cf-2-0_0_0_0_1_0',
 'cg-2-0_0_0_0_1_0',
 'ci-2-0_0_0_0_1_0',
 'cm-2-0_0_0_0_1_0',
 'dj-2-0_0_0_0_1_0',
 'er-2-0_0_0_0_1_0',
 'et-2-0_0_0_0_1_0',
 'fm-2-0_0_0_0_1_0',
 'ga-2-0_0_0_0_1_0',
 'gm-2-0_0_0_0_1_0',
 'gn-2-0_0_0_0_1_0',
 'gq-2-0_0_0_0_1_0',
 'gw-2-0_0_0_0_1_0',
 'hn-2-0_0_0_0_1_0',
 'ht-2-0_0_0_0_1_0',
 'ke-2-0_0_0_0_1_0',
 'kh-2-0_0_0_0_1_0',
 'km-2-0_0_0_0_1_0',
 'lr-2-0_0_0_0_1_0',
 'ls-2-0_0_0_0_1_0',
 'mg-2-0_0_0_0_1_0',
 'ml-2-0_0_0_0_1_0',
 'mm-2-0_0_0_0_1_0',
 'mr-2-0_0_0_0_1_0',
 'mz-2-0_0_0_0_1_0',
 'na-2-0_0_0_0_1_0',
 'ne-2-0_0_0_0_1_0',
 'ng-2-0_0_0_0_1_0',
 'ni-2-0_0_0_0_1_0',
 'pg-2-0_0_0_0_1_0',
 'pk-2-0_0_0_0_1_0',
 'rw-2-0_0_0_0_1_0',
 'sb-2-0_0_0_0_1_0',
 'sd-2-0_0_0_0_1_0',
 'sl-2-0_0_0_0_1_0',
 'sn-2-0_0_0_0_1_0',
 'so-2-0_0_0_0_1_0',
 'ss-2-0_0_0_0_1_0',
 'st-2-0_0_0_

In [123]:
# This will run for a single country, but could be looped over as is done above
country = 'zw-2'
xx_so = GEP.gep_results(country[:4], 
                scenariosFolder=scenario_base,
                clustersFolder = cluster_folder,
                scenario=scenario)
res = xx_so.join_results()
res['perElec'] = (res['ElecPopCalib'] / res['Pop2020'])

In [155]:
sel_res = res.loc[:,['id','ElecStart','ElecStatusIn2025','ElecStatusIn2030','CurrentMVLineDist','perElec','geometry']]
# Set the default value (already electrified)
sel_res['year_elec'] = 2019
# Select features that are electrified between 2020 and 2025
first = sel_res.loc[(sel_res['ElecStart'] == 0) & (sel_res['ElecStatusIn2025'] == 1)]
first = first.sort_values([sort_column]) #Sort by the predictive column
first['year_elec'] = get_years(first.shape[0], 2020, 2025)
# Select features that are electrified between 2025 and 2030
second = sel_res.loc[(sel_res['ElecStatusIn2025'] == 0) & (sel_res['ElecStatusIn2030'] == 1)]
second = second.sort_values([sort_column])
second['year_elec'] = get_years(second.shape[0], 2025, 2030)

sel_res.loc[first.index, 'year_elec'] = first['year_elec']
sel_res.loc[second.index, 'year_elec'] = second['year_elec']

sel_res['year_elec'] = sel_res['year_elec'] - sel_res['year_elec'].min() + 1
outFile = f"/media/gost/DATA1/GEP/temp/{country}_electrification_evolution_{sort_column}.tif"

#Before rasterizing, sort by year of electrification; I am not convinced this does anything
sel_res = sel_res.sort_values(['year_elec'], ascending=sort_final_year)
xx_so.rasterize_results(sel_res, outFile, res=resolution, field='year_elec')